In [2]:
import numpy as np
import cv2
import json
import glob
import os
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import glob
import json
import tensorflow as tf
from scipy.linalg import svd
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from scipy import spatial
from sklearn.metrics import accuracy_score
import datetime
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
for gpu in tf.config.list_physical_devices('GPU'):
    print(gpu) 

def RGB2Bayer (image):
    def isp_it(img):
        bayer_norm = (img/255).astype(np.float32)
        black_level = 6/255
        white_level = 68/255
        bayer_norm = (bayer_norm - black_level)/(white_level - black_level)
        bayer_norm = bayer_norm*white_level
        bayer_norm = np.clip(bayer_norm, 0.0, 1.0)
        # red channel
        bayer_norm[1::2, 1::2] = bayer_norm[1::2, 1::2]
        # blue channel
        bayer_norm[::2, ::2] = bayer_norm[::2, ::2]*1.25
        bayer_norm = bayer_norm*2.5
        bayer_norm = np.clip(bayer_norm, 0, 1)
        bayer = np.array(bayer_norm*255, dtype=np.uint8)
        img_rgb = cv2.cvtColor(bayer, cv2.COLOR_BAYER_RG2BGR)
        return img_rgb
    
    def reverse_isp_it(img_rgb):
        # Assume img_rgb is the output of the isp_it function
        # Since we cannot exactly reverse cv2.cvtColor, we start after this step
    
        # Constants used in the forward process
        black_level = 6 / 255
        white_level = 68 / 255
    
        # Reverse the scaling and clipping (from the end of isp_it2)
        img_rev = img_rgb.astype(np.float32) / 255
        img_rev = np.clip(img_rev, 0, 1) / 2.5
    
        # Reverse the blue channel adjustment
        img_rev[::2, ::2] = img_rev[::2, ::2] / 1.25
    
        # Reverse the normalization
        img_rev = img_rev / white_level
        img_rev = (img_rev * (white_level - black_level)) + black_level
    
        # Assume the output is a Bayer pattern image (but this is a rough approximation)
        bayer_approx = np.clip(img_rev * 255, 0, 255).astype(np.uint8)
        return bayer_approx
    
    def approximate_bayer_pattern(img_rgb):
        # Assuming img_rgb is the output from reverse_isp_it2
    
        # Initialize an empty array for the single-channel image
        height, width, _ = img_rgb.shape
        bayer_pattern = np.zeros((height, width), dtype=np.uint8)
    
        # Approximate the Bayer pattern based on RGGB pattern
        bayer_pattern[1::2, 1::2] = img_rgb[1::2, 1::2, 0]  # Red channel
        bayer_pattern[::2, ::2] = img_rgb[::2, ::2, 2]    # Blue channel
    
        # Green channel is interleaved between Red and Blue
        bayer_pattern[::2, 1::2] = img_rgb[::2, 1::2, 1]   # Green channel (first row)
        bayer_pattern[1::2, ::2] = img_rgb[1::2, ::2, 1]   # Green channel (second row)
        return bayer_pattern
    
    
    def convert_rgb_to_bayer(image):
        img_rgb = cv2.imread(image, -1)
        #img_rgb = cv2.resize(img_rgb, (324,324))    
        img_rgb = np.array(255*((img_rgb/255)**(1/0.65)), dtype='uint8')
        bayer_approx = reverse_isp_it(img_rgb)
        bayer = approximate_bayer_pattern(bayer_approx)
        return bayer
    return convert_rgb_to_bayer(image) 

def Bayer2RGB(bayer):
    def isp_it2(img):
        bayer_norm = (img/255).astype(np.float32)
        black_level = 6/255
        white_level = 68/255
        bayer_norm = (bayer_norm - black_level)/(white_level - black_level)
        bayer_norm=bayer_norm*white_level
        bayer_norm=np.clip(bayer_norm,0.0,1.0)
        #red channel
        bayer_norm[1::2,1::2]=bayer_norm[1::2,1::2]
        #blue channel
        bayer_norm[::2,::2]=bayer_norm[::2,::2]*1.25
        bayer_norm=bayer_norm*2.5 # was 2.5
        bayer_norm= np.clip(bayer_norm,0,1)
        bayer=np.array(bayer_norm*255,dtype=np.uint8)
        img_rgb = cv2.cvtColor(bayer, cv2.COLOR_BAYER_RG2RGB)
        return img_rgb
    linear_img=isp_it2(bayer)
    img_gammad=np.array(255*(linear_img/255)**0.65,dtype='uint8')
    return img_gammad

def show (img, scale_percent = 100, waitKey=-1):
    w = int(img.shape[1] * scale_percent / 100)
    h = int(img.shape[0] * scale_percent / 100)
    image = img.copy()
    img = cv2.resize(img, (w, h), interpolation = cv2.INTER_AREA)
    cv2.namedWindow("image", cv2.WINDOW_NORMAL)
    cv2.imshow('image', img)
    k = cv2.waitKey(waitKey) & 0xFF
    if k == ord('s'):
        cv2.imwrite("image.png", image)
        cv2.destroyAllWindows()     
    if k == ord('q'):
        cv2.destroyAllWindows()  
        return -1
        # 
def draw_bbox (image, od, label):
    img = image.copy()
    h,w = img.shape[:2]
    x1 = int(od[0])
    y1 = int(od[1])
    x2 = int(od[2])
    y2 = int(od[3])
        
    if label == "anchor":
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255),2)
        cv2.putText(img,label, org=(50,50),fontFace=2,fontScale=1, color=(255,255,255))
    if label == "positive":
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0),2)
        cv2.putText(img,label, org=(50,50),fontFace=2,fontScale=1, color=(0,255,0))
    if label == "negative":
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255),2)
        cv2.putText(img,label, org=(50,50),fontFace=2,fontScale=1, color=(0,0,255))
    return img

Num GPUs Available:  0


In [3]:
# Making feature dataset

# interpreter = tf.lite.Interpreter("custom_retinanet_mobilenetv1_pretrained_rot_fixed.tflite",experimental_preserve_all_tensors=True,num_threads=30)
# interpreter.allocate_tensors()
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()
# input_shape = input_details[0]['shape']
# scale = input_details[0]['quantization_parameters']['scales'][0]
# zero_point = input_details[0]['quantization_parameters']['zero_points'][0]

# dataset_path = "triplet-dataset/"
# folder_paths = [folder_name for folder_name in glob.glob(os.path.join(dataset_path,"*/"))]

# bbox_dataset = []
# rgb_dataset = []
# bayer_dataset = []
# feature_dataset = [] 

# for i in tqdm(range(len(folder_paths))):
#     # Getting Anchor Data
#     anchor_path = folder_paths[i]+"anchor"
#     anchor_rgb_path = glob.glob(anchor_path+"/*.png")[0]
#     anchor_bayer = RGB2Bayer(anchor_rgb_path)
#     anchor_rgb = Bayer2RGB(anchor_bayer)
#     with open(glob.glob(anchor_path+"/*.json")[0], 'r') as file:
#             anchor_od = json.load(file)
#     anchor_bbox = np.asarray(list(anchor_od.values())[1:])*anchor_bayer.shape[0]
#     anchor_bbox[0],anchor_bbox[1], anchor_bbox[2],anchor_bbox[3] = anchor_bbox[1],anchor_bbox[0], anchor_bbox[3], anchor_bbox[2]

#     positive_path = folder_paths[i]+"positive"
#     positive_rgb_path = glob.glob(positive_path+"/*.png")[0]
#     positive_bayer = RGB2Bayer(positive_rgb_path)
#     positive_rgb = Bayer2RGB(positive_bayer)
#     with open(glob.glob(positive_path+"/*.json")[0], 'r') as file:
#             positive_od = json.load(file)
#     positive_bbox = np.asarray(list(positive_od.values())[1:])*positive_bayer.shape[0]
#     positive_bbox[0],positive_bbox[1], positive_bbox[2],positive_bbox[3] = positive_bbox[1],positive_bbox[0], positive_bbox[3], positive_bbox[2]

#     negative_path = folder_paths[i]+"negative"
#     negative_rgb_path = glob.glob(negative_path+"/*.png")[0]
#     negative_bayer = RGB2Bayer(negative_rgb_path)
#     negative_rgb = Bayer2RGB(negative_bayer)
#     with open(glob.glob(negative_path+"/*.json")[0], 'r') as file:
#             negative_od = json.load(file)
#     negative_bbox = np.asarray(list(negative_od.values())[1:])*negative_bayer.shape[0]
#     negative_bbox[0],negative_bbox[1], negative_bbox[2],negative_bbox[3] = negative_bbox[1],negative_bbox[0], negative_bbox[3], negative_bbox[2]

#     bbox_dataset.append([anchor_bbox,positive_bbox,negative_bbox])
#     rgb_dataset.append([anchor_rgb,positive_rgb,negative_rgb])
#     bayer_dataset.append([anchor_bayer,positive_bayer,negative_bayer])

#     anchor_bayer = anchor_bayer / scale
#     anchor_bayer = anchor_bayer + zero_point
#     anchor_bayer = tf.cast(anchor_bayer,dtype=np.int8)
#     anchor_bayer  = tf.expand_dims(anchor_bayer,0)
#     anchor_bayer  = tf.expand_dims(anchor_bayer,3)
#     interpreter.set_tensor(input_details[0]['index'], anchor_bayer)
#     interpreter.invoke()
#     anchor_feature_a = interpreter.get_tensor(158)
#     anchor_feature_a = tf.cast(tf.image.resize(anchor_feature_a,(40,40),method='bilinear'),dtype = np.int8)
#     anchor_feature_b = interpreter.get_tensor(171)
#     anchor_feature_b = tf.cast(tf.image.resize(anchor_feature_b,(40,40),method='bilinear'),dtype = np.int8)
#     anchor_feature_c = interpreter.get_tensor(178)
#     anchor_feature_c = tf.cast(tf.image.resize(anchor_feature_c,(40,40),method='bilinear'),dtype = np.int8)
#     anchor_features = tf.concat([anchor_feature_a,anchor_feature_b,anchor_feature_c],3)
#     anchor_downscaled_box = np.round(anchor_bbox/8)
#     anchor_feature_box = anchor_features[:,int(anchor_downscaled_box[1]):int(anchor_downscaled_box[3]), int(anchor_downscaled_box[0]):int(anchor_downscaled_box[2])]
#     anchor_feature_box = tf.image.resize(anchor_feature_box,(10,10),method = 'bilinear')
#     anchor_feature_box = tf.squeeze(anchor_feature_box,0)
    
#     positive_bayer = positive_bayer / scale
#     positive_bayer = positive_bayer + zero_point
#     positive_bayer = tf.cast(positive_bayer,dtype=np.int8)
#     positive_bayer  = tf.expand_dims(positive_bayer,0)
#     positive_bayer  = tf.expand_dims(positive_bayer,3)
#     interpreter.set_tensor(input_details[0]['index'], positive_bayer)
#     interpreter.invoke()
#     positive_feature_a = interpreter.get_tensor(158)
#     positive_feature_a = tf.cast(tf.image.resize(positive_feature_a,(40,40),method='bilinear'),dtype = np.int8)
#     positive_feature_b = interpreter.get_tensor(171)
#     positive_feature_b = tf.cast(tf.image.resize(positive_feature_b,(40,40),method='bilinear'),dtype = np.int8)
#     positive_feature_c = interpreter.get_tensor(178)
#     positive_feature_c = tf.cast(tf.image.resize(positive_feature_c,(40,40),method='bilinear'),dtype = np.int8)
#     positive_features = tf.concat([positive_feature_a,positive_feature_b,positive_feature_c],3)
#     positive_downscaled_box = np.round(positive_bbox/8)
#     positive_feature_box = positive_features[:,int(positive_downscaled_box[1]):int(positive_downscaled_box[3]), int(positive_downscaled_box[0]):int(positive_downscaled_box[2])]
#     positive_feature_box = tf.image.resize(positive_feature_box,(10,10),method = 'bilinear')
#     positive_feature_box = tf.squeeze(positive_feature_box,0)
    
#     negative_bayer = negative_bayer / scale
#     negative_bayer = negative_bayer + zero_point
#     negative_bayer = tf.cast(negative_bayer,dtype=np.int8)
#     negative_bayer  = tf.expand_dims(negative_bayer,0)
#     negative_bayer  = tf.expand_dims(negative_bayer,3)
#     interpreter.set_tensor(input_details[0]['index'], negative_bayer)
#     interpreter.invoke()
#     negative_feature_a = interpreter.get_tensor(158)
#     negative_feature_a = tf.cast(tf.image.resize(negative_feature_a,(40,40),method='bilinear'),dtype = np.int8)
#     negative_feature_b = interpreter.get_tensor(171)
#     negative_feature_b = tf.cast(tf.image.resize(negative_feature_b,(40,40),method='bilinear'),dtype = np.int8)
#     negative_feature_c = interpreter.get_tensor(178)
#     negative_feature_c = tf.cast(tf.image.resize(negative_feature_c,(40,40),method='bilinear'),dtype = np.int8)
#     negative_features = tf.concat([ negative_feature_a, negative_feature_b, negative_feature_c],3)
#     negative_downscaled_box = np.round(negative_bbox/8)
#     negative_feature_box = negative_features[:,int(negative_downscaled_box[1]):int(negative_downscaled_box[3]), int(negative_downscaled_box[0]):int(negative_downscaled_box[2])]
#     negative_feature_box = tf.image.resize(negative_feature_box,(10,10),method = 'bilinear')
#     negative_feature_box = tf.squeeze(negative_feature_box,0)
    
#     feature_dataset.append([anchor_feature_box,positive_feature_box,negative_feature_box])

# np.save("feature_dataset.npy",feature_dataset)
# np.save("bayer_dataset.npy",bayer_dataset)
# np.save("rgb_dataset.npy",rgb_dataset)
# np.save("bbox_dataset.npy",bbox_dataset)

feature_dataset = np.load("feature_dataset.npy")
bayer_dataset = np.load("bayer_dataset.npy")
rgb_dataset = np.load("rgb_dataset.npy")
bbox_dataset = np.load("bbox_dataset.npy")


In [4]:
# Spliting the triplet dataset into train, test and val sets
X_train, X_val = train_test_split(feature_dataset,test_size=0.2,train_size=0.8,shuffle=True,random_state=40)
X_val, X_test = train_test_split(X_val,test_size=0.5,train_size=0.5,shuffle=True,random_state=40)

bbx_train, bbx_val = train_test_split(bbox_dataset,test_size=0.2,train_size=0.8,shuffle=True,random_state=40)
bbx_val, bbx_test = train_test_split(bbx_val,test_size=0.5,train_size=0.5,shuffle=True,random_state=40)

rgb_train, rgb_val = train_test_split(rgb_dataset,test_size=0.2,train_size=0.8,shuffle=True,random_state=40)
rgb_val, rgb_test = train_test_split(rgb_val,test_size=0.5,train_size=0.5,shuffle=True,random_state=40)

bayer_train, bayer_val = train_test_split(bayer_dataset,test_size=0.2,train_size=0.8,shuffle=True,random_state=40)
bayer_val, bayer_test = train_test_split(bayer_val,test_size=0.5,train_size=0.5,shuffle=True,random_state=40)

In [5]:
# checking the dataset
rgb_show = rgb_train.copy()
bbx_show = bbx_train.copy()
for index in range(len(rgb_show)):
    anchor_rgb = rgb_show[index][0]
    anchor_bbox = bbx_show[index][0]
    anchor_rgb = draw_bbox(anchor_rgb, anchor_bbox,label="anchor")
    positive_rgb = rgb_show[index][1]
    positive_bbox = bbx_show[index][1]
    positive_rgb = draw_bbox(positive_rgb, positive_bbox,label="positive")
    negative_rgb = rgb_show[index][2]
    negative_bbox = bbx_show[index][2]
    negative_rgb = draw_bbox(negative_rgb, negative_bbox,label="negative")
    # flag = show(np.hstack((anchor_rgb,positive_rgb,negative_rgb)))
    # if flag == -1:
    #     break
print(len(X_train),len(X_val),len(X_test))

2007 251 251


In [12]:
class CreateDistanceModel:
    def __init__(self, learning_rate=1e-3,decay = None, margin = 0.5 , height = 10, width = 10, channel = 1792):
        self.learning_rate = learning_rate
        self.margin = margin
        self.H = height
        self.W = width
        self.CH = channel
        self.decay = decay
        
    def distance(self,x,y):
        x = K.l2_normalize(x, axis=-1)
        y = K.l2_normalize(y, axis=-1)
        distance = 1 - K.batch_dot(x, y, axes=-1)
        return distance
        
    def triplet_loss(self,templates):
        anchor,positive,negative = templates
        positive_distance = self.distance(anchor,positive)
        negative_distance = self.distance(anchor,negative)
        basic_loss = positive_distance-negative_distance+self.margin
        loss = K.maximum(basic_loss,0.0)
        return loss

    def identity_loss(self,y_true, y_pred):
        return K.mean(y_pred)

    def __call__(self): 
        H,W,CH = (self.H, self.W, self.CH) # feature map size
        INPUT = layers.Input(shape =(H,W,CH))
        # X = layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(INPUT)
        # X=layers.UnitNormalization()(INPUT)
        # X = layers.Normalization()(INPUT)
        X = layers.Conv2D(8,(1,1),padding='same',kernel_initializer='normal',activation='relu')(INPUT)
        X = layers.Dropout(0.2)(X)
        X = layers.Flatten()(X)
        X = layers.Dense(1024,activation = 'relu')(X)
        X = layers.Dense(256,activation = 'relu')(X)
        Embedding = Model(INPUT, X, name="Embedding")
        anchor = layers.Input(shape=(H,W,CH), name='anchor_input')
        A = Embedding(anchor)
        positive = layers.Input(shape=(H,W,CH), name='positive_input')
        P = Embedding(positive)
        negative = layers.Input(shape=(H,W,CH), name='negative_input')
        N = Embedding(negative)
        LOSS = layers.Lambda(self.triplet_loss)([A, P, N])
        model = Model(inputs=[anchor,positive,negative],outputs=LOSS)
        model.compile(loss=self.identity_loss, optimizer=optimizers.Adamax(learning_rate=self.learning_rate, weight_decay = self.decay))
        return model

learning_rate = 0.001
epochs = 500
decay = learning_rate / epochs

DistClass = CreateDistanceModel(learning_rate=learning_rate,decay=None, margin=0.5, channel = 1792)
model = DistClass()
model.summary()
y_train, y_test, y_val = tf.ones(len(X_train)), tf.ones(len(X_test)), tf.ones(len(X_val))
callbacks=[EarlyStopping(patience=100, verbose=1, restore_best_weights=True, monitor='val_loss')]
history = model.fit([X_train[:,0],X_train[:,1],X_train[:,2]],y_train,epochs=epochs, batch_size=50,validation_data=([X_val[:,0],X_val[:,1],X_val[:,2]],y_val),callbacks=callbacks)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 anchor_input (InputLayer)   [(None, 10, 10, 1792)]       0         []                            
                                                                                                  
 positive_input (InputLayer  [(None, 10, 10, 1792)]       0         []                            
 )                                                                                                
                                                                                                  
 negative_input (InputLayer  [(None, 10, 10, 1792)]       0         []                            
 )                                                                                                
                                                                                            

In [8]:
# # summarize history for accuracy
# plt.figure(figsize=(8,7))
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train_loss','val_loss'], loc='center')
# plt.ylim([0, 0.5])
# plt.show()

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def draw_bbox_with_similarity_score (image, od, label,score):
    img = image.copy()
    h,w = img.shape[:2]
    x1 = int(od[0])
    y1 = int(od[1])
    x2 = int(od[2])
    y2 = int(od[3])
        
    if label == "anchor":
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255),2)
        cv2.putText(img,label, org=(50,50),fontFace=1,fontScale=1, color=(255,255,255))
        cv2.putText(img,str(score), org=(20,300),fontFace=1,fontScale=1, color=(255,255,255))
    if label == "positive":
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0),2)
        cv2.putText(img,label, org=(50,50),fontFace=1,fontScale=1, color=(0,255,0))
        cv2.putText(img,str(score), org=(200,300),fontFace=1,fontScale=1, color=(0,255,0))
    if label == "negative":
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255),2)
        cv2.putText(img,label, org=(50,50),fontFace=1,fontScale=1, color=(0,0,255))
        cv2.putText(img,str(score), org=(200,300),fontFace=1,fontScale=1, color=(0,0,255))
    return img

def analyze_model(model, X,rgb_dataset,bbox_dataset,show_flag=True):
    X_anchor = X[:,0]
    X_positive = X[:,1]
    X_negative = X[:,2]
    siamese_network = model.layers[3]
    X_anchor_template = siamese_network.predict(X_anchor)
    X_positive_template = siamese_network.predict(X_positive)
    X_negative_template = siamese_network.predict(X_negative)
    
    y_predict_targets_positive = []
    for index in range(len(X)):
        similarity = cosine_similarity(X_anchor_template[index].reshape(1, -1),X_positive_template[index].reshape(1, -1))
        y_predict_targets_positive.append(similarity[0][0])

    y_predict_targets_negative = []
    for index in range(len(X)):
        similarity = cosine_similarity(X_anchor_template[index].reshape(1, -1),X_negative_template[index].reshape(1, -1))
        y_predict_targets_negative.append(similarity[0][0])

    y_predict_targets_positive = np.array(y_predict_targets_positive)
    y_predict_targets_negative = np.array(y_predict_targets_negative)
    if show_flag== True:
        for index in range(len(rgb_dataset)):
            anchor_rgb = rgb_dataset[index][0]
            anchor_bbox = bbox_dataset[index][0]
            anchor_rgb = draw_bbox_with_similarity_score(anchor_rgb, anchor_bbox,label="anchor", score = "Cosine Similarity")
            
            positive_rgb = rgb_dataset[index][1]
            positive_bbox = bbox_dataset[index][1]
            positive_rgb = draw_bbox_with_similarity_score(positive_rgb, positive_bbox,label="positive", score = y_predict_targets_positive[index])
        
            negative_rgb = rgb_dataset[index][2]
            negative_bbox = bbox_dataset[index][2]
            negative_rgb = draw_bbox_with_similarity_score(negative_rgb, negative_bbox,label="negative", score = y_predict_targets_negative[index])
        
            flag = show(np.hstack((anchor_rgb,positive_rgb,negative_rgb)))
            if flag == -1:
                break
    y_targets_positive = np.ones((len(X),))
    y_targets_negative = np.ones((len(X),))
    y_p=(y_predict_targets_positive>=0.5).astype(np.uint8)
    y_n=(y_predict_targets_negative<0.5).astype(np.uint8)
    return (accuracy_score(y_targets_positive,y_p)+accuracy_score(y_targets_negative,y_n))/2
     
# analyze_model(model, X_val,rgb_val,bbx_val, show_flag=False)    

8/8 [==============================] - 0s 4ms/step


0.8884462151394422

In [13]:
# analyze_model(model, X_test,rgb_test,bbx_test, show_flag=False)    

In [13]:
# analyze_model(model, X_train,rgb_train,bbx_train, show_flag=False)    

In [42]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(X_train[:,0]).batch(1).take(100):
    yield [input_value]
siamese_network = model.layers[3]
converter = tf.lite.TFLiteConverter.from_keras_model(siamese_network)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmphyepyvrj/assets


INFO:tensorflow:Assets written to: /tmp/tmphyepyvrj/assets
/home/shah/.local/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-03-19 12:59:04.348002: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-03-19 12:59:04.348028: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-03-19 12:59:04.348206: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmphyepyvrj
2024-03-19 12:59:04.349240: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-03-19 12:59:04.349255: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmphyepyvrj
2024-03-19 12:59:04.351790: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-03-19 12:59:04.3701

In [43]:
import pathlib
tflite_models_dir = pathlib.Path("tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
tflite_model_quant_file = tflite_models_dir/"EmbeddingModel_MobNet_FeatureBased2.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

560944

In [44]:
import pathlib
tflite_models_dir = pathlib.Path("tflite_models/")
tflite_model_quant_file = tflite_models_dir/"EmbeddingModel_MobNet_FeatureBased2.tflite"
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter.allocate_tensors()
interpreter.get_input_details()[0]

{'name': 'serving_default_input_2:0',
 'index': 0,
 'shape': array([   1,   10,   10, 1792], dtype=int32),
 'shape_signature': array([  -1,   10,   10, 1792], dtype=int32),
 'dtype': numpy.int8,
 'quantization': (1.0, 0),
 'quantization_parameters': {'scales': array([1.], dtype=float32),
  'zero_points': array([0], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [45]:
interpreter.get_output_details()[0]

{'name': 'StatefulPartitionedCall:0',
 'index': 11,
 'shape': array([  1, 256], dtype=int32),
 'shape_signature': array([ -1, 256], dtype=int32),
 'dtype': numpy.int8,
 'quantization': (0.9768680930137634, -128),
 'quantization_parameters': {'scales': array([0.9768681], dtype=float32),
  'zero_points': array([-128], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [46]:
debugger = tf.lite.experimental.QuantizationDebugger(converter=converter, debug_dataset = representative_data_gen)
debugger.run()

INFO:tensorflow:Assets written to: /tmp/tmpwnsq_28t/assets


INFO:tensorflow:Assets written to: /tmp/tmpwnsq_28t/assets
/home/shah/.local/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-03-19 12:59:51.470795: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-03-19 12:59:51.470819: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-03-19 12:59:51.471000: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpwnsq_28t
2024-03-19 12:59:51.471956: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-03-19 12:59:51.471972: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpwnsq_28t
2024-03-19 12:59:51.474074: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-03-19 12:59:51.4933

In [47]:
debugger.layer_statistics

defaultdict(<function tensorflow.lite.tools.optimize.debugging.python.debugger.QuantizationDebugger._collect_layer_statistics.<locals>.<lambda>()>,
            {'NumericVerify/Embedding/conv2d_1/Relu;Embedding/conv2d_1/BiasAdd;Embedding/conv2d_1/Conv2D;Embedding/conv2d_1/BiasAdd/ReadVariableOp1:16': defaultdict(list,
                         {'num_elements': 800.0,
                          'stddev': 1.7730403,
                          'mean_error': -0.16651128,
                          'max_abs_error': 7.2843866,
                          'mean_squared_error': 3.3062944}),
             'NumericVerify/Embedding/flatten/Reshape1:20': defaultdict(list,
                         {'num_elements': 800.0,
                          'stddev': 1.962091e-06,
                          'mean_error': -7.898808e-08,
                          'max_abs_error': 1.4801026e-05,
                          'mean_squared_error': 3.9889585e-12}),
             'NumericVerify/Embedding/dense/MatMul;Embedding/d